
# In short: Handful of files that contains all the key omic information from hundred thousands of sequencing runs
Skymap is a standalone database that offers: 
1. **a single data matrix** for each omic layer for each species that [spans >200k sequencing runs from all the public studies](https://www.ncbi.nlm.nih.gov/sra), which is done by reprocessing **petabytes** worth of sequencing data. Here is how much published data are out there: 
![alt text](./Figures/sra_data_availability.png "Logo Title Text 1")

2. **a biological metadata file** that describe the relationships between the sequencing runs and also the keywords extracted from over **3 million** freetext annotations using NLP. 
3. **a techinical metadata file** that describe the relationships between the sequencing runs. 


**Where they can all fit into your personal computer.**

** If you intend to run the examples, please first download the data from here:** https://www.synapse.org/skymap (take < 3 minutes to set up the account). 

Table of Contents
=================

* [In long](#in-long)
  * [Motivation: Pooling processed data from multiple studies is time\-consuming:](#motivation-pooling-processed-data-from-multiple-studies-is-time-consuming)
  * [Solution: An automated pipeline to generate a single data matrix that does simple counting for each specie and omic layer](#solution-an-automated-pipeline-to-generate-a-single-data-matrix-that-does-simple-counting-for-each-specie-and-omic-layer)
  * [Why Skymap while there are so many groups out there also trying to unify the public data](#why-skymap-while-there-are-so-many-groups-out-there-also-trying-to-unify-the-public-data)
  * [Why Skymap offer a local copy instead of a web api](#why-skymap-offer-a-local-copy-instead-of-a-web-api)
  * [Data format and coding style](#data-format-and-coding-style)
* [Data slicing example](#data-slicing-example)
    * [Accessing allelic read count dataframe](#accessing-allelic-read-count-dataframe)
    * [Accessing RNAseq dataframe](#accessing-rnaseq-dataframe)
    * [Accesing biological metadata dataframe](#accesing-biological-metadata-dataframe)
    * [Accessing technical metadata dataframe](#accessing-technical-metadata-dataframe)
* [More examples on using simple code to analyze big data](#more-examples-on-using-simple-code-to-analyze-big-data)
  * [High resolution mouse developmental hierachy map](#high-resolution-mouse-developmental-hierachy-map)
  * [Locating  SNP and correlating with different data layers](#locating--snp-and-correlating-with-different-data-layers)
  * [Simple RNAseq data slicing and hypothesis testing](#simple-rnaseq-data-slicing-and-hypothesis-testing)
  * [Acknowledgement](#acknowledgement)




# In long
## Motivation: Pooling processed data from multiple studies is time-consuming: 
When I first started in bioinformatic couple years ago, I spent much of my time doing two things: 1.) cleaning omic data matrices, e.g. mapping between gene IDs (hgnc, enseml, ucsc, etc.) for processed data matrices, trying all sort of different bioinformatics pipelines that yield basically the same results, investigating what is the exact unit being counted over when pulling pre-processed data from public database, etc.  2.) cleaning metadata annotation, which usually involves extracting and aliasing the labels to the exact same categories. 


This question came to my mind: Can we merge and reduce the petabytes worth of raw reads into a single table that: 1.) captures the commonly used information which can 2.) also fit in your hard drive (<500Gb)? 

## Solution: An automated pipeline to generate a single data matrix that does simple counting for each specie and omic layer 
What I am offering in here is a metadata table and a single data matrix for each omic layer that encapsulate majority of the public data out there by continuously pulling data from [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra), a place that host the bulk of the published sequencing data. I believe that “Science started with counting” (from “Cancer: Emperor of all malady” by Siddhartha Mukherjee), and thus I offer counts for all the features: 1. ) the  base resolution ACGT counts for over 200k experiments among NCBI curated SNPs, where read depth and allelic fraction are usually the main drivers for SNP calling. We also offer an expression matrix, which counts at both transcript and gene resolution. With the raw counts, you can normalize however you want. 
The metadata table consists of controlled vocabulary (NCI Terminology) from free text experiment annotations. I used the NLM metamap engine for extracting keywords from freetext. The nice thing is that the UMLS ecosystem from NLM allow the IDs (Concept Unique Identifiers) to be mapped onto different ontologies to relate the terms. NCIT is by far the cleanest general purpose ontology I have seen, low term redundancy, encode medical knowledge from many domains and is well maintained.  
[The pipeline in here](https://github.com/brianyiktaktsui/Skymap/tree/master/code) is trying to suit the needs of the common use cases. In another word, most pipelines out there are more like sport cars, having custom flavors for a specific group of drivers. What I am trying to create is more like a train system, aiming to suit most needs. Unfortunately, if you have more specific requirements, what I am offering is probably not going to work. 


Here are the overview slides for the overall processes of [allelic read counts extraction over 300k known SNPs](https://docs.google.com/presentation/d/1KcumgtLfCdHNnIwkbU5DaQ7UNKHGbJ_fJZFy1cj53yE/edit#slide=id.p3), [RNAseq quantification and NLP processing](https://docs.google.com/presentation/d/14vLJJQ6ziw-2aLDoQAJGyv1sYo5ENzljsqsbZr9jNLM/edit#slide=id.p19), explaining 1.) why the data is something that you can trust and 2.) also the utility of fast data interpolation, which is especially useful for aggregating multiple studies/batches to support your hypothesis. 

## Why Skymap while there are so many groups out there also trying to unify the public data
To the best of my knowledge, Skymap is the first that offer both the unified omic data and the cleaned metadata. The other important aspect is that the process of data extraction is fully automated, so it is supposed to be scalable and systematic. 

## Why Skymap offer a local copy instead of a web api 
Again, the purpose of this project is more geared towards bioinformatics/ data scientists, who wants go from vast amount of data to hypothesis quickly. I hate when I have to recover a simple table by requesting each row from REST api repeately, which should have only required one click on an ftp link. It turns out that even [all the raw meta data from SRA can fit into memory](https://github.com/brianyiktaktsui/Skymap/blob/master/Load_RawMetaData.ipynb). 

The premise of skymap is this: Couple clicks and all the omic data sits in your computer. And you can slice and dice it however you want efficiently. 

## Data format and coding style

The storage is in python pandas pickle format. Therefore, the only packges you need to load in the data is numpy and pandas, the backbone of data analysis in python. We keep the process of data loading as lean as possible. Less code means less bugs and less errors. For now, Skymap is geared towards ML/data science folks who are hungry for the vast amount of data and ain’t afraid of coding. I will port the data to native HDF5 format to reduce platform dependency once I get a chance. 

I tried to keep the code and parameters to be lean and self-explanatory for your reference. 


# Data slicing example

### Accessing allelic read count dataframe
Slice out >100k experiments and their allelic counts in < 1s

In [1]:
### parameters
import pandas as pd
import numpy as np
mySpecie='Homo_sapiens'
#change base dir to your data location
baseDir='/cellar/users/btsui/Data/SRA/snp/'
skymap_snp_dir=baseDir+'{specie}_snp_pos/'.format(specie=mySpecie)

**input query BRAF V600 coordinate**

In [2]:
#location where BRAF V600 happens, you can change it to whatever position you want 
queryChr,queryPosition='7',140753336 

**static code for slicing out the data**

In [3]:
%%time
chunkSize=100000 #fixed params
myChunk=(queryPosition/chunkSize)*chunkSize # identify the chunk to load in
hdf_s=pd.HDFStore(skymap_snp_dir+'Pos_block_'+str(myChunk),mode='r')#load in the chunk
tmpChunkDf=hdf_s['/chunk'] 

CPU times: user 112 ms, sys: 132 ms, total: 244 ms
Wall time: 303 ms


In [4]:
print '# of sequencing runs sliced out:' ,tmpChunkDf.Run_digits.nunique()

# of sequencing runs sliced out: 149064


** definition of each column in allelic counts data**

Chr: Chromosome

Base: DNA bases in aligned reads - A, C, G, T 

Run_db and Run_digits together forms a SRR accession id. I ignored the leading 0s for Run_digits. 

ReadDepth: the number of bases detected in aligned reads at a particular base and chromosome position. 

AverageBaseQuality: The mean phred score in aligned reads at a particular base and chromosome postiion. 

Pos: Chromosome position. (grch38 for human)

block: the block ID used for chunked storage

In [5]:
tmpChunkDf.head()

features         Run_digits        Pos  ReadDepth  AverageBaseQuality  \
Chr base Run_db                                                         
2   C    SRR         796215  140700401          1                  41   
    A    SRR        5882370  140700401          1                  11   
    C    SRR        3420530  140700401        140                  36   
         SRR         586184  140700401          2                  35   
         SRR        4444531  140700401         16                  39   

features             block  
Chr base Run_db             
2   C    SRR     140700000  
    A    SRR     140700000  
    C    SRR     140700000  
         SRR     140700000  
         SRR     140700000

### Accessing RNAseq dataframe

Read out any gene expression level for >100k of experiments in 10ms. 

input: genename, output: experiment by TPM vector

In [6]:
"""
a standalone function for memory mapping the data
"""
def loadDf(fname,mmap_mode='r'):
    with open(fname+'.index.txt') as f:
        myIndex=map(lambda s:s.replace("\n",""), f.readlines())
    with open(fname+'.columns.txt') as f:
        myColumns=map(lambda s:s.replace("\n",""), f.readlines())
    tmpMatrix=np.load(fname+".npy",mmap_mode=mmap_mode)
    tmpDf=pd.DataFrame(tmpMatrix,index=myIndex,columns=myColumns)
    tmpDf.columns.name='Run'
    return tmpDf

In [8]:
expressionMetric='TPM'
#change this to where the matrix is located on your computer
baseDir='/cellar/users/btsui/Data/nrnb01_nobackup/Data/SRA/MATRIX/DATA/hgGRC38/'
dataMatrixDir=baseDir+'/allSRAmatrix.realign.v9.base.{feature}.gene.symbol'.format(feature=expressionMetric)

In [9]:
%%time
rnaseqDf=loadDf(dataMatrixDir)

CPU times: user 120 ms, sys: 24 ms, total: 144 ms
Wall time: 134 ms


In [10]:
%%time
geneS=rnaseqDf.loc['CDK1']

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 7.39 ms


In [12]:
geneS.head()

Run
SRR4456480    56.119999
SRR4456481     0.000000
SRR4456482     0.000000
SRR4456483     0.000000
SRR4456484     0.000000
Name: CDK1, dtype: float32

### Accesing biological metadata dataframe

For more information about bio_metaDf columns:

Sample: https://www.ncbi.nlm.nih.gov/books/NBK56913/

attribute: https://www.ncbi.nlm.nih.gov/biosample/docs/attributes/

NCIT_Eng, NCIT_ID: https://ncit.nci.nih.gov/

NLM_CUI: https://www.nlm.nih.gov/research/umls/new_users/online_learning/Meta_005.html

The NLP tool used for mapping freetexts to terms is called metamap:
https://metamap.nlm.nih.gov/

In [13]:
metaDataMappingSDir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP//input/allAttrib.v5.csv.NCI.prefilter.pyc'
bio_metaDf=pd.read_pickle(metaDataMappingSDir)

Here is how it looks like

In [17]:
bio_metaDf.head()

,srs,attrib,CUI,score,NCI,NciEng
0,SRS286232,sex,C1706180,1000,C46109,Male Gender
1,SRS286232,sex,C1706429,1000,C46107,"Male, Self-Report"
2,SRS286232,sex,C1706428,1000,C46112,Male Phenotype
3,DRS052357,BioSampleModel,C1332821,694,C24597,CXCL9 Gene
4,DRS052357,BioSampleModel,C1707170,694,C49770,CXCL9 wt Allele


In [14]:
print '# of unique biological sample annotations with terms extracted:',bio_metaDf['srs'].nunique()

# of unique biological sample annotations with terms extracted: 3068221


Millions of biological annotations have NLP key words extracted with high number of unique terms, suggesting that public data deposited in SRA has both high volumne and diversity in experimental conditions. 

In [15]:
print '# of unique biomedical terms:',bio_metaDf['NCI'].nunique()

# of unique biomedical terms: 20150


### Accessing technical metadata dataframe

For more information about the aliases used in the follow meta data:

https://www.ncbi.nlm.nih.gov/books/NBK56913/



In [18]:
##change the directory
sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.pickle'
technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)

In [21]:
print '# of SRRs: ',technical_meta_data_df.shape[0]

# of SRRs:  3763299


In [20]:
technical_meta_data_df.head()

,Member_Name,Experiment,Sample,Study,Spots,Bases,Status,ScientificName,LibraryStrategy,LibraryLayout,...,proj_accession_Updated,proj_accession_Published,proj_accession_Received,proj_accession_Type,proj_accession_Center,proj_accession_Visibility,proj_accession_Loaded,proj_accession_ReplacedBy,Run_db,Run_digits
Run,,,,,,,,,,,,,,,,,,,,,
SRR2401865,default,SRX1244330,SRS1068422,-,2800.0,1416405.0,live,soil_metagenome,AMPLICON,SINGLE,...,2015-09-22,2015-09-20,2015-09-15,RUN,SUB1095135,public,1,-,SRR,2401865
SRR2401866,default,SRX1244331,SRS1068421,-,5082.0,2563605.0,live,soil_metagenome,AMPLICON,SINGLE,...,2015-09-22,2015-09-20,2015-09-15,RUN,SUB1095135,public,1,-,SRR,2401866
SRR2401867,default,SRX1244332,SRS1068420,-,6169.0,3175528.0,live,soil_metagenome,AMPLICON,SINGLE,...,2015-09-22,2015-09-20,2015-09-15,RUN,SUB1095135,public,1,-,SRR,2401867
SRR2401868,default,SRX1244333,SRS1068419,-,8102.0,4266915.0,live,soil_metagenome,AMPLICON,SINGLE,...,2015-09-22,2015-09-20,2015-09-15,RUN,SUB1095135,public,1,-,SRR,2401868
SRR2401869,default,SRX1244334,SRS1068418,-,4971.0,2519200.0,live,soil_metagenome,AMPLICON,SINGLE,...,2015-09-22,2015-09-20,2015-09-15,RUN,SUB1095135,public,1,-,SRR,2401869


# More examples on using simple code to analyze big data

**If you intend to run the example notebooks, first download the data from synapse**

https://www.synapse.org/#!Synapse:syn11415602/wiki/492470


## High resolution mouse developmental hierachy map
[Link](https://github.com/brianyiktaktsui/Skymap/blob/master/jupyter-notebooks/clean_notebooks/TemporalQuery_V4_all_clean.ipynb
)

Aggregating many studies (node) to form a smooth mouse developmental hierachy map. By integrating the vast amount of public data, we can cover many developmental time points, which sometime we can see a more transient expression dynamics both across tissues and within tissues over developmental time course. 

Each componenet represent a tissue. Each node represent a particular study at a particular time unit. The color is base on the developmental time extracted from experimental annotation using regex. The node size represent the # of sequencing runs in that particulr time point and study. Each edge represent a differentiate-to or part-of relationship.
![alt text](./Figures/heirachy_time.png "Logo Title Text 1")
And you can easily overlay gene expression level on top of it. As an example, Tp53 expression is known to be tightly regulated in development. Let's look at the dynamic of Tp53 expression over time and spatial locations in the following plot.
![alt_text](./Figures/heirachy_Trp53.png "tp53")

## Locating  SNP and correlating with different data layers
https://github.com/brianyiktaktsui/Skymap/blob/master/FindStudiesWithBrafV600Mutated.ipynb
## Simple RNAseq data slicing and hypothesis testing
https://github.com/brianyiktaktsui/Skymap/blob/master/DataSlicingExample.ipynb

[Check here for more example notebooks](https://github.com/brianyiktaktsui/Skymap/tree/master/jupyter-notebooks
)

The code for the pipelines is here:
https://github.com/brianyiktaktsui/Skymap/tree/master/code

Skymap is still in Beta V0.0. [Please feel free to leave comments](https://www.synapse.org/#!Synapse:syn11415602/discussion/default) and suggestions!!! We would love to hear feedbacks from you.
## Acknowledgement


Please considering citing if you are using Skymap. (doi:10.7303/syn11415602)

Acknowledgement: We want to thank for the advice and resources from Dr. Hannah Carter (my PI), Dr. Jill Mesirov,Dr. Trey Ideker and Shamin Mollah. We also want to thank Dr. Ruben Arbagayen, Dr. Nate Lewis for their suggestion. 
The method will soon be posted in bioarchive. Also, we want to thank the Sage Bio Network for hosting the data. We also thank to thank the NCBI for holding all the published raw reads at  [Sequnece Read Archive](https://www.ncbi.nlm.nih.gov/sra). 
Grant money that make this work possible: NIH DP5OD017937,GM103504

Term of use: Use Skymap however you want. Just dont sue me, I have no money. 

For why I named it Skymap, I forgot.

In [1]:
!ls /cellar/users/btsui/Data/SRA/DUMP/

FULL_SRA_meta				file_meta.txt
NCBI_SRA_Metadata_Full_20180702.tar.gz	merged_variant_aligning_statistics.tsv
SRA_Accessions.tab			meta.txt
SRA_Run_Members.tab			sra_dump.csv.gz
allSRS.pickle.gz			sra_dump.fastqc.bowtie_algn.pickle
allSRX.pickle.gz			sra_dump.pickle
